## Term Project     DSC - 540

In this project I am trying to scrape ZIPcodes from www.zip-codes.com for North Carolina and use those ZIPcodes to pull current weather data from https://openweathermap.org/current and do some data transformations to display data in readable format. 

In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import pprint
import datetime
import time

### Scrape ZIP codes from https://www.zip-codes.com/state/nc.asp

In [2]:
# use get method to send a GET request to the URL
page = requests.post('https://www.zip-codes.com/state/nc.asp')

#passes the HTML of the page into the BeautifulSoup class
bs = BeautifulSoup(page.content)

#Uses Beautiful Soup to find and return all tables with id containing the string "tblZIP"
_tables = bs.find_all("table", id ="tblZIP")
print(len(_tables))
print(_tables)

1
[<table border="0" cellpadding="0" cellspacing="0" class="statTable" id="tblZIP" title="All North Carolina ZIP Codes, City, County, Classification, and Area Codes." width="99%">
<tr>
<td class="label" title="All ZIP Codes for North Carolina"><strong>ZIP Code</strong></td>
<td class="info" title="The official city name as designated by the USPS."><strong>City</strong></td>
<td class="info" title="The primary county or parish this ZIP Code serves."><strong>County</strong></td>
<td class="info" title="The classification type of this ZIP Code."><strong>Type</strong></td>
</tr><tr><td><a href="/zip-code/27006/zip-code-27006.asp" title="ZIP Code 27006">ZIP Code 27006</a></td><td><a href="/city/nc-advance.asp" title="Advance, NC">Advance</a></td><td><a href="/county/nc-davie.asp">Davie</a></td><td>Standard</td></tr><tr><td><a href="/zip-code/27007/zip-code-27007.asp" title="ZIP Code 27007">ZIP Code 27007</a></td><td><a href="/city/nc-ararat.asp" title="Ararat, NC">Ararat</a></td><td><a href

In [3]:
# create zipcodeslist
_zipcodeList = []

# find all a's in the first table
_AList = _tables[0].find_all('a')

# extract 'title' for all a's
for _a in _AList:
    if 'title' in _a.attrs.keys():
        _title = _a['title']
        
        # extract zipcodes string and append zipcodelist
        if _title.startswith('ZIP'):
            _zipcodeList.append(_title.split(' ')[-1])

# find length of the zipcodelist and print
print(len(_zipcodeList))
print(_zipcodeList)


1083
['27006', '27007', '27009', '27010', '27011', '27012', '27013', '27014', '27016', '27017', '27018', '27019', '27020', '27021', '27022', '27023', '27024', '27025', '27027', '27028', '27030', '27031', '27040', '27041', '27042', '27043', '27045', '27046', '27047', '27048', '27049', '27050', '27051', '27052', '27053', '27054', '27055', '27094', '27098', '27099', '27101', '27102', '27103', '27104', '27105', '27106', '27107', '27108', '27109', '27110', '27111', '27113', '27114', '27115', '27116', '27117', '27120', '27127', '27130', '27150', '27152', '27155', '27157', '27198', '27199', '27201', '27202', '27203', '27204', '27205', '27207', '27208', '27209', '27212', '27213', '27214', '27215', '27216', '27217', '27228', '27229', '27230', '27231', '27233', '27235', '27237', '27239', '27242', '27243', '27244', '27247', '27248', '27249', '27252', '27253', '27256', '27258', '27259', '27260', '27261', '27262', '27263', '27264', '27265', '27268', '27278', '27281', '27282', '27283', '27284', '272

In [4]:
# finding duplicates in zipcodelist
# convert zipcodelist to set
x=set(_zipcodeList)
# create duplicates list
dup=[]
# find and append dulicates list
for c in x:
    if(_zipcodeList.count(c)>1):
        dup.append(c)
# print duplicate ZIP codes list
print(dup)

['28301', '27709', '27711']


In [5]:
# create final list of zipcodes with out duplicates and find length of the list
finalList = list(set(_zipcodeList)) 
len(finalList)

1080

### Pull current weather data from https://openweathermap.org/current and do some data transformations to display data in readable format.

In [8]:
# define get_weather function to extract data from API
def get_weather(code):
    API_KEY = '303edfe18e79163b5aa9cea46e5b8e65'
    url_base = 'http://api.openweathermap.org/data/2.5/weather?'
    url = url_base+'zip='+code+',us&appid='+API_KEY+'&units=imperial'
    r = requests.get(url)
    data = r.json()
    return data

# create used zip codes list and weather data list
used_list = []
weather_data = []

# extract data from API using each zip code from list
for zc in finalList[:len(finalList)]:
    # if zip code is not in used list perform next steps and append the used zipcode list
    if zc not in used_list :
        used_list.append(zc)        
        # try to get weather data from API and append weather_data list
        try:
            data1 = get_weather(zc)
            # wait for 0.2 sec to move to another step to limit number of calls per minute 
            time.sleep(0.2)
            weather_data.append(data1)
            print(data1)
            
        # prints message to user if unable to open url
        except requests.exceptions.ConnectionError as errc:
            # handle ConnectionError exception
            print('\033[91m ' +'***Connection Failure. Please try later.***'+'\033[0m')
            break

        # handle all other exceptions
        except Exception as e:
            print('\033[91m '+"Failure to Retrieve.Please try again"+'\033[0m')


{'coord': {'lon': -77.85, 'lat': 35.99}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 59.61, 'feels_like': 60.13, 'temp_min': 57, 'temp_max': 62.01, 'pressure': 1011, 'humidity': 93}, 'visibility': 16093, 'wind': {'speed': 3.36, 'deg': 160}, 'clouds': {'all': 90}, 'dt': 1582677348, 'sys': {'type': 1, 'id': 4012, 'country': 'US', 'sunrise': 1582631254, 'sunset': 1582671710}, 'timezone': -18000, 'id': 0, 'name': 'Rocky Mount', 'cod': 200}
{'coord': {'lon': -83.07, 'lat': 35.26}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 51.46, 'feels_like': 48.94, 'temp_min': 46, 'temp_max': 57.2, 'pressure': 1012, 'humidity': 81}, 'visibility': 16093, 'wind': {'speed': 2.73, 'deg': 264}, 'clouds': {'all': 1}, 'dt': 1582677504, 'sys': {'type': 1, 'id': 2708, 'country': 'US', 'sunrise': 1582632465, 'sunset': 1582673005}, 'timezone': -18000, '

{'coord': {'lon': -79.18, 'lat': 35.48}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 59.16, 'feels_like': 58.75, 'temp_min': 57, 'temp_max': 63, 'pressure': 1011, 'humidity': 100}, 'visibility': 1207, 'wind': {'speed': 6.02, 'deg': 204}, 'clouds': {'all': 90}, 'dt': 1582677508, 'sys': {'type': 1, 'id': 6072, 'country': 'US', 'sunrise': 1582631544, 'sunset': 1582672058}, 'timezone': -18000, 'id': 0, 'name': 'Sanford', 'cod': 200}
{'coord': {'lon': -83.22, 'lat': 35.07}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 51.03, 'feels_like': 48.2, 'temp_min': 45, 'temp_max': 57.2, 'pressure': 1012, 'humidity': 81}, 'visibility': 16093, 'wind': {'speed': 3.11, 'deg': 307}, 'clouds': {'all': 1}, 'dt': 1582677508, 'sys': {'type': 1, 'id': 2708, 'country': 'US', 'sunrise': 1582632490, 'sunset': 1582673051}, 'timezone': -18000, 'id': 0, 'name': 'High

{'coord': {'lon': -81.38, 'lat': 35.25}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 56.17, 'feels_like': 54.57, 'temp_min': 51.8, 'temp_max': 60.01, 'pressure': 1010, 'humidity': 82}, 'visibility': 16093, 'wind': {'speed': 3.36, 'deg': 220}, 'clouds': {'all': 40}, 'dt': 1582677512, 'sys': {'type': 1, 'id': 3954, 'country': 'US', 'sunrise': 1582632059, 'sunset': 1582672600}, 'timezone': -18000, 'id': 0, 'name': 'Kings Mountain', 'cod': 200}
{'coord': {'lon': -81.9, 'lat': 36.07}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 51.19, 'feels_like': 47.3, 'temp_min': 46, 'temp_max': 55.99, 'pressure': 1011, 'humidity': 81}, 'visibility': 16093, 'wind': {'speed': 5.06, 'deg': 332}, 'clouds': {'all': 100}, 'dt': 1582677512, 'sys': {'type': 1, 'id': 6048, 'country': 'US', 'sunrise': 1582632230, 'sunset': 1582672679}, 'timezo

{'coord': {'lon': -82.29, 'lat': 35.9}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 52.25, 'feels_like': 50.05, 'temp_min': 46.99, 'temp_max': 55.99, 'pressure': 1011, 'humidity': 87}, 'visibility': 16093, 'wind': {'speed': 3.36, 'deg': 300}, 'clouds': {'all': 90}, 'dt': 1582677287, 'sys': {'type': 1, 'id': 2666, 'country': 'US', 'sunrise': 1582632314, 'sunset': 1582672782}, 'timezone': -18000, 'id': 0, 'name': 'Burnsville', 'cod': 200}
{'coord': {'lon': -77.77, 'lat': 35.31}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 62.22, 'feels_like': 61.99, 'temp_min': 59, 'temp_max': 66.2, 'pressure': 1010, 'humidity': 93}, 'visibility': 16093, 'wind': {'speed': 6.33, 'deg': 192}, 'clouds': {'all': 90}, 'dt': 1582677517, 'sys': {'type': 1, 'id': 4267, 'country': 'US', 'sunrise': 1582631197, 'sunset': 1582671729}, 'timezone': -18000,

{'coord': {'lon': -80.37, 'lat': 35.76}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 54.97, 'feels_like': 54.12, 'temp_min': 51.01, 'temp_max': 59, 'pressure': 1011, 'humidity': 100}, 'visibility': 16093, 'wind': {'speed': 4.25, 'deg': 212}, 'clouds': {'all': 40}, 'dt': 1582677521, 'sys': {'type': 1, 'id': 4031, 'country': 'US', 'sunrise': 1582631846, 'sunset': 1582672329}, 'timezone': -18000, 'id': 0, 'name': 'Linwood', 'cod': 200}
{'coord': {'lon': -82.57, 'lat': 35.39}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 52.52, 'feels_like': 48.74, 'temp_min': 46, 'temp_max': 59, 'pressure': 1010, 'humidity': 76}, 'visibility': 16093, 'wind': {'speed': 4.7, 'deg': 170}, 'clouds': {'all': 1}, 'dt': 1582677305, 'sys': {'type': 1, 'id': 3351, 'country': 'US', 'sunrise': 1582632352, 'sunset': 1582672878}, 'timezone': -18000, 'id': 

...

{'coord': {'lon': -82.2, 'lat': 35.25}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 53.62, 'feels_like': 49.68, 'temp_min': 48, 'temp_max': 59, 'pressure': 1011, 'humidity': 71}, 'visibility': 16093, 'wind': {'speed': 4.7, 'deg': 250}, 'clouds': {'all': 1}, 'dt': 1582677786, 'sys': {'type': 1, 'id': 5896, 'country': 'US', 'sunrise': 1582632256, 'sunset': 1582672797}, 'timezone': -18000, 'id': 0, 'name': 'Columbus', 'cod': 200}
{'coord': {'lon': -80.44, 'lat': 36.33}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 52.39, 'feels_like': 51.58, 'temp_min': 48, 'temp_max': 55.4, 'pressure': 1010, 'humidity': 87}, 'visibility': 16093, 'wind': {'speed': 0.94, 'deg': 214}, 'clouds': {'all': 1}, 'dt': 1582677786, 'sys': {'type': 1, 'id': 5842, 'country': 'US', 'sunrise': 1582631895, 'sunset': 1582672313}, 'timezone': -18000, 'id': 0, 'name': 

{'coord': {'lon': -75.9, 'lat': 36.3}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 60.04, 'feels_like': 58.82, 'temp_min': 55.99, 'temp_max': 62.6, 'pressure': 1011, 'humidity': 100}, 'visibility': 16093, 'wind': {'speed': 8.03, 'deg': 226}, 'clouds': {'all': 75}, 'dt': 1582677790, 'sys': {'type': 1, 'id': 4082, 'country': 'US', 'sunrise': 1582630804, 'sunset': 1582671224}, 'timezone': -18000, 'id': 0, 'name': 'Aydlett', 'cod': 200}
{'coord': {'lon': -80.26, 'lat': 36.04}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 53.87, 'feels_like': 52.68, 'temp_min': 51.01, 'temp_max': 57.99, 'pressure': 1011, 'humidity': 100}, 'visibility': 16093, 'wind': {'speed': 4.25, 'deg': 212}, 'clouds': {'all': 40}, 'dt': 1582677539, 'sys': {'type': 1, 'id': 4031, 'country': 'US', 'sunrise': 1582631835, 'sunset': 1582672286}, 'timezone': 

{'coord': {'lon': -77.94, 'lat': 34.75}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 62.92, 'feels_like': 62.67, 'temp_min': 57.99, 'temp_max': 66.2, 'pressure': 1011, 'humidity': 88}, 'visibility': 16093, 'wind': {'speed': 5.82, 'deg': 180}, 'clouds': {'all': 40}, 'dt': 1582677795, 'sys': {'type': 1, 'id': 4038, 'country': 'US', 'sunrise': 1582631207, 'sunset': 1582671801}, 'timezone': -18000, 'id': 0, 'name': 'Wallace', 'cod': 200}
{'coord': {'lon': -78.78, 'lat': 35.76}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 58.96, 'feels_like': 58.51, 'temp_min': 55.99, 'temp_max': 62.01, 'pressure': 1011, 'humidity': 93}, 'visibility': 16093, 'wind': {'speed': 4.7, 'deg': 220}, 'clouds': {'all': 90}, 'dt': 1582677795, 'sys': {'type': 1, 'id': 5645, 'country': 'US', 'sunrise': 1582631464, 'sunset': 1582671947}, 'timezone':

{'coord': {'lon': -77.91, 'lat': 36.05}, 'weather': [{'id': 300, 'main': 'Drizzle', 'description': 'light intensity drizzle', 'icon': '09n'}], 'base': 'stations', 'main': {'temp': 59.32, 'feels_like': 58.62, 'temp_min': 57, 'temp_max': 61, 'pressure': 1011, 'humidity': 100}, 'visibility': 11265, 'wind': {'speed': 6.62, 'deg': 197}, 'clouds': {'all': 90}, 'dt': 1582677799, 'sys': {'type': 1, 'id': 4296, 'country': 'US', 'sunrise': 1582631272, 'sunset': 1582671721}, 'timezone': -18000, 'id': 0, 'name': 'Red Oak', 'cod': 200}
{'coord': {'lon': -79.82, 'lat': 34.91}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 61.79, 'feels_like': 62.46, 'temp_min': 57, 'temp_max': 64.99, 'pressure': 1011, 'humidity': 88}, 'visibility': 16093, 'wind': {'speed': 3.47, 'deg': 195}, 'clouds': {'all': 40}, 'dt': 1582677799, 'sys': {'type': 1, 'id': 5636, 'country': 'US', 'sunrise': 1582631666, 'sunset': 1582672244}, 'timezon

{'coord': {'lon': -80.8, 'lat': 35.26}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 56.71, 'feels_like': 55.74, 'temp_min': 54, 'temp_max': 60.01, 'pressure': 1010, 'humidity': 93}, 'visibility': 16093, 'wind': {'speed': 4.32, 'deg': 204}, 'clouds': {'all': 75}, 'dt': 1582677641, 'sys': {'type': 1, 'id': 4001, 'country': 'US', 'sunrise': 1582631921, 'sunset': 1582672460}, 'timezone': -18000, 'id': 0, 'name': 'Charlotte', 'cod': 200}
{'coord': {'lon': -76.47, 'lat': 36.19}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 60.3, 'feels_like': 58.84, 'temp_min': 55.4, 'temp_max': 64.4, 'pressure': 1012, 'humidity': 93}, 'visibility': 16093, 'wind': {'speed': 7.27, 'deg': 218}, 'clouds': {'all': 90}, 'dt': 1582677804, 'sys': {'type': 1, 'id': 3934, 'country': 'US', 'sunrise': 1582630935, 'sunset': 1582671368}, 'timezone': -1800

In [9]:
len(weather_data)

1080

In [10]:
len(used_list)

1080

In [11]:
# using == to check if lists are equal 
if used_list == finalList: 
    print ("The lists are identical") 
else : 
    print ("The lists are not identical") 

The lists are identical


In [12]:
# create 'weather.json' file with weather data
with open('weather.json', 'w') as outfile:
    json.dump(weather_data, outfile)

In [13]:
# read json file
with open('weather.json', 'r') as myfile:
    data = myfile.read()

    # parse file
    weather_data1 = json.loads(data)
    
    # using pprint to print each record
    # output will be a dictonary for each record
    for item in weather_data1:
        pprint.pprint(item) 


{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 35.99, 'lon': -77.85},
 'dt': 1582677348,
 'id': 0,
 'main': {'feels_like': 60.13,
          'humidity': 93,
          'pressure': 1011,
          'temp': 59.61,
          'temp_max': 62.01,
          'temp_min': 57},
 'name': 'Rocky Mount',
 'sys': {'country': 'US',
         'id': 4012,
         'sunrise': 1582631254,
         'sunset': 1582671710,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 160, 'speed': 3.36}}
{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 35.26, 'lon': -83.07},
 'dt': 1582677504,
 'id': 0,
 'main': {'feels_like': 48.94,
          'humidity': 81,
          'pressure': 1012,
          'temp': 51.46,
          'temp_max': 57.2,
          'temp_min': 46},
 'name': 'Tuckasegee',
 'sys': {'country':

         'id': 6048,
         'sunrise': 1582632234,
         'sunset': 1582672689,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 332, 'speed': 5.06}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 34.7, 'lon': -76.74},
 'dt': 1582677514,
 'id': 0,
 'main': {'feels_like': 59.34,
          'humidity': 93,
          'pressure': 1012,
          'temp': 60.06,
          'temp_max': 63,
          'temp_min': 57},
 'name': 'Atlantic Beach',
 'sys': {'country': 'US',
         'id': 4973,
         'sunrise': 1582630916,
         'sunset': 1582671516,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 240, 'speed': 5.82}}
{'base': 'sta

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 35.36, 'lon': -79.11},
 'dt': 1582677524,
 'id': 0,
 'main': {'feels_like': 60.15,
          'humidity': 100,
          'pressure': 1011,
          'temp': 60.19,
          'temp_max': 64.4,
          'temp_min': 57},
 'name': 'Olivia',
 'sys': {'country': 'US',
         'id': 6072,
         'sunrise': 1582631521,
         'sunset': 1582672048,
         'type': 1},
 'timezone': -18000,
 'visibility': 1207,
 'weather': [{'description': 'mist', 'icon': '50n', 'id': 701, 'main': 'Mist'}],
 'wind': {'deg': 204, 'speed': 6.02}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 34.73, 'lon': -76.75},
 'dt': 1582677324,
 'id': 0,
 'main': {'feels_like': 59.2,
          'humidity': 93,
          'pressure': 1012,
          'temp': 59.95,
          'temp_max': 63,
          'temp_min': 57},
 'name': 'Morehead City',
 'sys': {'country': 'US',
         'id': 4973,
         'sunrise': 1582630920,


              'main': 'Clouds'}],
 'wind': {'deg': 220, 'speed': 3.36}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 35.96, 'lon': -77.3},
 'dt': 1582677535,
 'id': 0,
 'main': {'feels_like': 60.4,
          'humidity': 93,
          'pressure': 1011,
          'temp': 59.81,
          'temp_max': 62.6,
          'temp_min': 57.2},
 'name': 'Oak City',
 'sys': {'country': 'US',
         'id': 4012,
         'sunrise': 1582631121,
         'sunset': 1582671580,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 160, 'speed': 3.36}}
{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 35.28, 'lon': -81.21},
 'dt': 1582677535,
 'id': 0,
 'main': {'feels_like': 54.86,
          'humidity': 93,
          'pressure': 1010,
          'temp': 56.3,
          'temp_max': 59,
    

          'temp_min': 54},
 'name': 'Charlotte',
 'sys': {'country': 'US',
         'id': 4001,
         'sunrise': 1582631922,
         'sunset': 1582672482,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 234, 'speed': 4.59}}
{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 35.01, 'lon': -80.56},
 'dt': 1582677432,
 'id': 0,
 'main': {'feels_like': 55.63,
          'humidity': 93,
          'pressure': 1010,
          'temp': 57.34,
          'temp_max': 61,
          'temp_min': 54},
 'name': 'Monroe',
 'sys': {'country': 'US',
         'id': 4001,
         'sunrise': 1582631849,
         'sunset': 1582672416,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main':

...

          'pressure': 1011,
          'temp': 57.81,
          'temp_max': 61,
          'temp_min': 53.6},
 'name': 'Henderson',
 'sys': {'country': 'US',
         'id': 4393,
         'sunrise': 1582631405,
         'sunset': 1582671823,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 200, 'speed': 4.7}}
{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 35.61, 'lon': -80.44},
 'dt': 1582677753,
 'id': 0,
 'main': {'feels_like': 54.97,
          'humidity': 87,
          'pressure': 1011,
          'temp': 56.12,
          'temp_max': 60.01,
          'temp_min': 53.01},
 'name': 'Granite Quarry',
 'sys': {'country': 'US',
         'id': 5726,
         'sunrise': 1582631854,
         'sunset': 1582672354,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'cle

          'pressure': 1011,
          'temp': 59.14,
          'temp_max': 63,
          'temp_min': 57},
 'name': 'Cumnock',
 'sys': {'country': 'US',
         'id': 5891,
         'sunrise': 1582631542,
         'sunset': 1582672057,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 190, 'speed': 5.82}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 35.91, 'lon': -81.54},
 'dt': 1582677470,
 'id': 0,
 'main': {'feels_like': 51.01,
          'humidity': 93,
          'pressure': 1012,
          'temp': 51.84,
          'temp_max': 57,
          'temp_min': 46},
 'name': 'Lenoir',
 'sys': {'country': 'US',
         'id': 4971,
         'sunrise': 1582632135,
         'sunset': 1582672601,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clo

 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 214, 'speed': 0.94}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 36.09, 'lon': -81.88},
 'dt': 1582677780,
 'id': 0,
 'main': {'feels_like': 47.59,
          'humidity': 87,
          'pressure': 1011,
          'temp': 51.06,
          'temp_max': 55.99,
          'temp_min': 46},
 'name': 'Pineola',
 'sys': {'country': 'US',
         'id': 2666,
         'sunrise': 1582632226,
         'sunset': 1582672673,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 340, 'speed': 5.08}}
{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 35.23, 'lon': -80.84},
 'dt': 1582677774,
 'id': 0,
 'main': {'feels_like': 55.74,
      

          'temp_min': 53.6},
 'name': 'Bahama',
 'sys': {'country': 'US',
         'id': 4393,
         'sunrise': 1582631513,
         'sunset': 1582671951,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 200, 'speed': 4.7}}
{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 35.45, 'lon': -83.29},
 'dt': 1582677792,
 'id': 0,
 'main': {'feels_like': 45.79,
          'humidity': 62,
          'pressure': 1012,
          'temp': 50.31,
          'temp_max': 55.99,
          'temp_min': 45},
 'name': 'Whittier',
 'sys': {'country': 'US',
         'id': 2762,
         'sunrise': 1582632528,
         'sunset': 1582673047,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear

         'sunset': 1582671452,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 224, 'speed': 6.42}}
{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 35.26, 'lon': -80.8},
 'dt': 1582677641,
 'id': 0,
 'main': {'feels_like': 55.74,
          'humidity': 93,
          'pressure': 1010,
          'temp': 56.71,
          'temp_max': 60.01,
          'temp_min': 54},
 'name': 'Charlotte',
 'sys': {'country': 'US',
         'id': 4001,
         'sunrise': 1582631921,
         'sunset': 1582672460,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 204, 'speed': 4.32}}
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'

In [14]:
# define extract_details_and_print function
def extract_details_and_print(raw_json_data):

    # extracting location variable from data
    location = raw_json_data["name"]

    # store the value of "main" key in variable y
    y = raw_json_data["main"]

    # store the value of "temp" in current_temperature
    current_temperature = y["temp"]

    # store the value of "pressure" in current_pressure
    current_pressure = y["pressure"]

    # store the value of "humidity" in current_humidiy
    current_humidiy = y["humidity"]

    # store the value of "weather" key in variable z
    z = raw_json_data["weather"]

    # store the value corresponding to the "description" key at the 0th index of z as weather_description
    weather_description = z[0]["description"]

    #extracting wind speed from data
    wind_speed = raw_json_data["wind"]["speed"]

    #extracting date, sunrise & sunset times
    date = raw_json_data["dt"]
    
    sunrise = raw_json_data["sys"]["sunrise"]
    
    sunset = raw_json_data["sys"]["sunset"] 

    # converting timestamp to human readable format and print
    
    Date_time = datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
    
    sunrise_time = datetime.datetime.fromtimestamp(sunrise).strftime('%H:%M')
    
    sunset_time = datetime.datetime.fromtimestamp(sunset).strftime('%H:%M')
    
    print(" Date & Time:", '\33[32m ' + Date_time +'\033[0m')    
                                                                                                                
    print(" Sunrise: {} \t \t Sunset : {}".format('\33[34m'+ sunrise_time+'\033[0m', '\33[34m'+ sunset_time +'\033[0m'))                                                                                       
    
    # print the following values
    print(  " Location : " + '\33[34m' + str(location) + '\033[0m'
            "\n Temperature (in Fahrenheit) : " +
            '\33[34m' + str(current_temperature) + '\033[0m' +
            "\n Atmospheric pressure (in hPa) : " +
            '\33[34m' + str(current_pressure) + '\033[0m' +
            "\n Humidity (in percentage) : " +
            '\33[34m'+ str(current_humidiy) + '\033[0m' +
            "\n Weather description : " +
            '\33[34m'+ str(weather_description)+ '\033[0m' +
            "\n Wind speed (in mi/hr) :" +
            '\33[34m'+ str(wind_speed)+ '\033[0m' +
            '\33[34m' + "\n****************************************" + '\033[0m')


In [15]:
# print weather data in readable format
print('\33[34m'+"***************************************"+
            "\n \t WEATHER INFORMATION" +
            "\n***************************************" + '\033[0m')

# print weather information for first 5 zipcodes
for wd in weather_data1[0:5]:    
    extract_details_and_print(wd)


***************************************
 	 WEATHER INFORMATION
***************************************
 Date & Time:  2020-02-25 19:35:48
 Sunrise: 06:47 	 	 Sunset : 18:01
 Location : Rocky Mount
 Temperature (in Fahrenheit) : 59.61
 Atmospheric pressure (in hPa) : 1011
 Humidity (in percentage) : 93
 Weather description : overcast clouds
 Wind speed (in mi/hr) :3.36
****************************************
 Date & Time:  2020-02-25 19:38:24
 Sunrise: 07:07 	 	 Sunset : 18:23
 Location : Tuckasegee
 Temperature (in Fahrenheit) : 51.46
 Atmospheric pressure (in hPa) : 1012
 Humidity (in percentage) : 81
 Weather description : clear sky
 Wind speed (in mi/hr) :2.73
****************************************
 Date & Time:  2020-02-25 19:36:20
 Sunrise: 06:50 	 	 Sunset : 18:04
 Location : Raleigh
 Temperature (in Fahrenheit) : 59.02
 Atmospheric pressure (in hPa) : 1011
 Humidity (in percentage) : 100
 Weather description : light rain
 Wind speed (in mi/hr) :5.77
**************************